<a href="https://colab.research.google.com/github/mgk0422/python-basic/blob/master/final_project_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sklearn pandas matplotlib graphviz

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
무_시장=pd.read_excel('/content/무_시장.xlsx')

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [ ]:
무_시장.head()

In [ ]:
x_data=무_시장[['반입량','경락가평균가격','유가 전국평균가격','서울평균도매가격']]
y_target=무_시장[['가격']]
x,y=x_data,y_target
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)

##### 타겟변수 확인

##### https://mjs1995.tistory.com/63

In [ ]:
from scipy import stats
from scipy.stats import norm

In [ ]:
sns.distplot(무_시장['가격'],fit=norm)

In [ ]:
stats.probplot(무_시장['가격'],plot=plt)

In [ ]:
무_시장['가격']=np.log1p(무_시장['가격'])
sns.distplot(무_시장['가격'],fit=norm)

In [ ]:
stats.probplot(무_시장['가격'], plot=plt)

#XGBoost

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [ ]:
xgbr.fit(xtrain, ytrain)

In [ ]:
score = xgbr.score(xtrain, ytrain)  
print("Training score: ", score)
scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())
ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plot_importance(xgbr)
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from

#RandomForestRegressor

In [ ]:
rf=RandomForestRegressor(random_state=0,n_estimators=1000)
neg_mse_scores=cross_val_score(rf,x_data,y_target,scoring='neg_mean_squared_error',cv=5)
rmse_scores=np.sqrt(-1*neg_mse_scores)
avg_rmse=np.mean(rmse_scores)

print('5교차의 개별 Negative MSE scores : ',np.round(neg_mse_scores,2))
print('5교차 검증의 개별 RMSE score : ',np.round(rmse_scores,2))
print('5교차 검증의 평균 RMSE : {0:3f}'.format(avg_rmse))

In [ ]:
def get_model_cv_prediction(model, x_data,y_target):
    neg_mse_scores=cross_val_score(model,x_data,y_target,scoring='neg_mean_squared_error',cv=5)
    rmse_scores=np.sqrt(-1*neg_mse_scores)
    avg_rmse=np.mean(rmse_scores)
    print('####',model.__class__.__name__,'####')
    print('5교차 검증의 평균 RMSE : {0:3f}'.format(avg_rmse))
    score = xgbr.score(xtrain, ytrain)  
    print("Training score: ", score)
    scores = cross_val_score(xgbr, xtrain, ytrain,cv=10)
    print("Mean cross-validation score: %.2f" % scores.mean())
    kfold = KFold(n_splits=10, shuffle=True)
    kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
    print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

# 다양한 유형의 회귀 트리를 생성하기

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

dt_reg=DecisionTreeRegressor(random_state=0,max_depth=4)
rf_reg=RandomForestRegressor(random_state=0,n_estimators=1000)
gb_reg=GradientBoostingRegressor(random_state=0,n_estimators=1000)
xgb_reg=XGBRegressor(n_estimators=1000)
lgb_reg=LGBMRegressor(n_estimators=1000)

# 트리기반의 회귀 모델을 반복하면서 평가 수행
models=[dt_reg,rf_reg,gb_reg,xgb_reg,lgb_reg]
for model in models:
    get_model_cv_prediction(model, x_data, y_target)

In [ ]:
import seaborn as sns
%matplotlib inline

rf_reg=RandomForestRegressor(n_estimators=1000)

rf_reg.fit(x_data, y_target)

feature_series=pd.Series(data=rf_reg.feature_importances_,index=x_data.columns)
feature_series=feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series,y=feature_series.index)

In [ ]:
import seaborn as sns
%matplotlib inline

xgb_reg=XGBRegressor(n_estimators=1000)

xgb_reg.fit(x_data, y_target)

feature_series=pd.Series(data=xgb_reg.feature_importances_,index=x_data.columns)
feature_series=feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series,y=feature_series.index)

In [ ]:
x_ax = range(len(ytest))
plt.plot(x_ax, ytest, label="original")
plt.plot(x_ax, ypred, label="predicted")
plt.title(" test and predicted data")
plt.legend()
plt.show()

# https://joyfuls.tistory.com/62 : 상관관계


# 모델

In [ ]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
무_시장['가격']=np.log1p(무_시장['가격'])
sns.distplot(무_시장['가격'],fit=norm)

In [ ]:
stats.probplot(무_시장['가격'], plot=plt)

# 그래디언트 부스팅 회귀 트리(GradientBoostingRegressor)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbrt = GradientBoostingRegressor(random_state=0)
gbrt.fit(xtrain,ytrain)

print("훈련 세트 정확도: {:.3f}".format(gbrt.score(xtrain,ytrain)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(xtrain,ytrain)))


In [ ]:
## 최대 깊이를 1로 줄이기

gbrt = GradientBoostingRegressor(random_state=0, max_depth=1)
gbrt.fit(xtrain,ytrain)

print("훈련 세트 정확도: {:.3f}".format(gbrt.score(xtrain,ytrain)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(xtrain,ytrain)))

In [ ]:
# 개별 ML 모델을 위한 Classifier 생성.
knn_clf  = KNeighborsClassifier(n_neighbors=4) #K최근접이웃
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)#랜덤포레스트
dt_clf = DecisionTreeClassifier() #결정트리
ada_clf = AdaBoostregre(n_estimators=100) #아다부스트